This file contains the necessary code to get the counts and metrics for each company and then runs it through the various ML models

In [22]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LinearRegression
import numpy as np
import csv
import os
import pandas as pd
import math
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow
import tflearn
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [5]:
dir = 'C:/Users/Chaitu Konjeti/socweb-glassdoor-project/REVIEWS/'

In [6]:
all_metrics = {}

metrics_file = pd.read_csv('metrics.csv')

# get all the proper metrics from the metrics file
for row in metrics_file.index:
    #print(int(metrics_file['Data Year - Fiscal'][row]))
    if metrics_file['Company Name'][row] not in all_metrics.keys():
        all_metrics[metrics_file['Company Name'][row]] = []
        if not math.isnan(metrics_file['Gross Profit (Loss)'][row]) and not math.isnan(int(metrics_file['Data Year - Fiscal'][row])) and not int(metrics_file['Data Year - Fiscal'][row]) != '0':
            all_metrics[metrics_file['Company Name'][row]].append((int(metrics_file['Gross Profit (Loss)'][row]), str(int(metrics_file['Data Year - Fiscal'][row]))))
    else:
        if not math.isnan(metrics_file['Gross Profit (Loss)'][row]) and not math.isnan(int(metrics_file['Data Year - Fiscal'][row])):
            all_metrics[metrics_file['Company Name'][row]].append((int(metrics_file['Gross Profit (Loss)'][row]), str(int(metrics_file['Data Year - Fiscal'][row]))))

C:\Users\Chaitu Konjeti\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [58]:
def get_sentiment(sentences):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(sentences)
#     print("{:-<65} {}".format(sentences, str(vs['compound'])))
    return vs['compound']

In [75]:
def get_counts_and_metrics(dir):

    metrics = []
    list_of_posts = []
    X = []
    sentiments = []

    #iterate through all files in the directory
    for filename in os.listdir(dir):
        print(filename)
        df = pd.read_csv(dir + filename, header=0)

        #get dates and pros columns from data file
        dates = df['date']
        pros = df['pros']

        #get the list of years for all the posts in the file and create a dictionary with years as the keys
        years = [date.split(' ')[3] for date in dates]
        corpus = {key: '' for key in set(years)}

        #add the post to the proper year in the dictionary
        for date, pro in zip(years, pros):
            corpus[date] += pro + ' '

        #sort the keys in order
        keys = sorted(corpus)

        company_name = filename.split('.')[0]

        metric_years = [val[1] for val in set(all_metrics[company_name]) if val[1] in years]

        metrics.extend([val[0] for val in set(all_metrics[company_name]) if val[1] in years])

        #only use the metrics from years that present in the datafile
        for key in keys:
            if key in metric_years:
                sentiments.append(get_sentiment(corpus[key]))
                list_of_posts.append(corpus[key])
#     print(len(sentiments))
#     print(len(list_of_posts))
    #creates count vectorizer for posts
    count_vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 3))
    counts = count_vectorizer.fit_transform(list_of_posts)

    #creates Tfidf vectorizer for posts
    tfid_vectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='word')
    tfid = tfid_vectorizer.fit_transform(list_of_posts)
    metrics = np.asarray(metrics)
    
    i = 0
    for vec, sentiment in zip(tfid, sentiments):
        X.extend(vec, sentiment)
        i += 1
    
    ###COMBINE SENTIMENTS AND TFID VECTORS
    X = np.asarray(X)
    
    return  X, metrics

In [76]:
X, y = get_counts_and_metrics(dir)

AMERICAN AIRLINES GROUP INC.csv
APPLE INC.csv
AVNET INC.csv


TypeError: extend() takes exactly one argument (2 given)

In [ ]:
print(X.shape)

In [74]:
print(y.shape)

(28,)


In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [68]:
rff_clf = RandomForestClassifier(max_depth=10)
rff_clf.fit(X_train, y_train)

C:\Users\Chaitu Konjeti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ValueError: setting an array element with a sequence.

In [69]:
rff_clf.score(X_train, y_train)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [70]:
print(rff_clf.score(X_test, y_test))

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [16]:
y_test

array([  3100,  71867, 104126,  78483,  71061,  14485])

In [17]:
lr_clf = LinearRegression()
lr_clf.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [18]:
lr_clf.score(X_train, y_train)

1.0

In [19]:
print(rff_clf.score(X_test, y_test))

0.0


In [20]:
lr_clf.predict(X_test)

array([26297.07523627, 26525.6055515 , 32618.92836369, 29429.07480672,
       24503.00553311, 21570.98238764])

In [21]:
y_test

array([  3100,  71867, 104126,  78483,  71061,  14485])